<a href="https://colab.research.google.com/github/hmh10098/d2l/blob/main/6_2_Convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install d2l

     |████████████████████████████████| 83 kB 1.6 MB/s 


In [2]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
def corr2d(X, K):
  h, w = K.shape
  Y = torch.zeros(X.shape[0] - h + 1, X.shape[1] - w + 1)
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
  return Y

In [5]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [7]:
class Conv2D(nn.Module):
  def __init__(self, kernel_size):
    super().__init()
    self.weight = nn.Parameter(torch.randn(kernel_size))
    self.bias = nn.Parameter(torch.zeros(1))
  
  def forward(self, x):
    return corr2d(x, self.weight) + self.bias

In [8]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [10]:
K = torch.tensor([[1.0, -1.0]])
Y = corr2d(X, K)

In [11]:
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [15]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [16]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias = False)

X = X.reshape((1, 1, 6 ,8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2

for i in range(10):
  y_hat = conv2d(X)
  l = (y_hat - Y) ** 2
  conv2d.zero_grad()
  l.sum().backward()
  conv2d.weight.data[:] -= lr*conv2d.weight.grad
  if i % 2:
    print(f'batch{i+1}, loss: {l.sum()}')

batch2, loss: 10.357911109924316
batch4, loss: 3.3376193046569824
batch6, loss: 1.2152589559555054
batch8, loss: 0.47229698300361633
batch10, loss: 0.18917909264564514


In [18]:
print(conv2d.weight.shape)

torch.Size([1, 1, 1, 2])


In [22]:
X = torch.tensor([[1, 0 , 0, 0, 0], [0, 1, 0, 0, 0], [0, 0 , 1, 0, 0], [0, 0, 0, 1, 0], [0, 0 , 0, 0, 1]])
X

tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]])

In [25]:
corr2d(X ,K.T)

tensor([[ 1., -1.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.],
        [ 0.,  0.,  1., -1.,  0.],
        [ 0.,  0.,  0.,  1., -1.]])